In [ ]:
from typing import Any, List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from db_connections.pg_sql import session, Ticket
import numpy as np


In [ ]:
EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'
EMBEDDING_DIM = 768
N_CLUSTERS = 2

In [30]:
sample_tickets = [
    {"ticket_id": "1", "text": "Cannot connect to the company VPN, it keeps saying authentication failed.", "category": "Network"},
    {"ticket_id": "2", "text": "My Microsoft Outlook is not opening, it crashes on startup.", "category": "Software"},
    {"ticket_id": "3", "text": "The new laptop I received won't boot up, it shows a black screen.", "category": "Hardware"},
    {"ticket_id": "4", "text": "I'm unable to access the shared drive, it says permission denied.", "category": "Network"},
    {"ticket_id": "5", "text": "Excel is freezing frequently when I work with large spreadsheets.", "category": "Software"},
    {"ticket_id": "6", "text": "The printer on the 3rd floor is not working, documents are stuck in queue.", "category": "Hardware"},
    {"ticket_id": "7", "text": "How do I reset my domain password?", "category": "Account"},
]

In [41]:
sample_tickets = [
    {"ticket_id": "1", "text": "Cannot connect to the company VPN, it keeps saying authentication failed.", "category": "Network"},
    {"ticket_id": "2", "text": "My Microsoft Outlook is not opening, it crashes on startup.", "category": "Software"},
]

In [ ]:
def process_and_store_to_pgvector(tickets: List[Dict[str, Any]], model: SentenceTransformer, kmeans: KMeans):
    texts = [ticket['text'] for ticket in tickets]
    embeddings = model.encode(texts, show_progress_bar=True)
    print(f"embedding shPE: {embeddings.shape}")
    cluster_ids = kmeans.fit_predict(embeddings)

    for i, ticket in enumerate(tickets):
        embedding = np.array(embeddings[i], dtype=np.float32)
        new_ticket = Ticket(
            ticket_id=ticket['ticket_id'],
            text=ticket['text'],
            embedding=embedding
        )
        
        session.add(new_ticket)

    session.commit()
    print(f"Inserted {len(tickets)} tickets into PostgreSQL with pgvector.")

In [32]:

def query_similar_from_pgvector(query: str, model: SentenceTransformer, top_k: int = 3):
    query_vector = np.array(model.encode([query])[0], dtype=np.float32)

    sql = f"""
    SELECT id, ticket_id, text,
           embedding <#> cube(:query_embedding) as distance
    FROM tickets
    ORDER BY embedding <#> cube(:query_embedding)
    LIMIT :top_k;
    """

    result = session.execute(
        sql,
        {"query_embedding": query_vector.tolist(), "top_k": top_k}
    )

    print("Search Results:")
    for row in result:
        print(f"ID: {row.id}, Ticket ID: {row.ticket_id}, Text: {row.text}, Distance: {row.distance:.4f}")


In [48]:
sbert_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
kmeans = KMeans(n_clusters=1, random_state=42, n_init='auto')
process_and_store_to_pgvector(sample_tickets, sbert_model, kmeans)

Batches: 100%|██████████| 1/1 [00:00<00:00, 40.87it/s]

embedding shPE: (2, 384)


PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (builtins.ValueError) expected 768 dimensions, not 384
[SQL: INSERT INTO tickets (ticket_id, text, embedding) SELECT p0::VARCHAR, p1::VARCHAR, p2::VECTOR(768) FROM (VALUES (%(ticket_id)s, %(text)s, %(embedding)s)) AS imp_sen(p0, p1, p2, sen_counter) ORDER BY sen_counter RETURNING tickets.id, tickets.id AS id__1]
[parameters: [{'embedding': array([-6.63655475e-02,  3.42290588e-02,  1.24685112e-02, -2.55581066e-02,
       -4.58992943e-02,  2.31425129e-02,  3.60830836e-02, -3. ... (7039 characters truncated) ... 45e-02,  4.56833746e-03],
      dtype=float32), 'text': 'Cannot connect to the company VPN, it keeps saying authentication failed.', 'ticket_id': '1'}, {'embedding': array([ 3.21427062e-02, -4.40553539e-02,  2.69629080e-02,  1.07551329e-02,
        1.67914126e-02, -3.765

In [42]:
texts = [ticket['text'] for ticket in sample_tickets]
embeddings = sbert_model.encode(texts, show_progress_bar=True)
cluster_ids = kmeans.fit_predict(embeddings)

Batches: 100%|██████████| 1/1 [00:00<00:00, 48.27it/s]


In [46]:
embeddings.shape

(2, 384)